In [11]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import re
import os

os.chdir('..')
os.chdir('results')

In [12]:
def get_path(dataset='FordA', model_name='ResidualCNN', reg=True, exp_name=None):
    attack_type = 'fgsm_disc_attack' if reg else 'fgsm_attack'
    exp = '_' + exp_name if exp_name else ''
    
    path = '/'.join([dataset, model_name, 'attack', attack_type + exp])

    return path



def combined_df(dataset='FordA', model_name='ResidualCNN', reg=True, exp_name=None):

    path = get_path(dataset=dataset, model_name=model_name, reg=reg, exp_name=exp_name)
    
    files = os.listdir(path)
    files = [f for f in files if f.endswith('.csv')]

    pattern = r"(?<==).*?(?=.csv)"
    id_pattern = fr"(?<={dataset}_).*?(?=_alpha)"

    all_res = pd.DataFrame()

    for file in files:
        curr_res = pd.read_csv(path + '/' + file, index_col=0)

        curr_res['alpha'] = float(re.search(pattern, file)[0])
        curr_res['model_id'] = re.search(id_pattern, file)[0]

        curr_res.columns = [column.strip(' ') for column in curr_res.columns] 

        all_res = pd.concat([all_res, curr_res], axis=0)

    return all_res.reset_index(drop=True)

def aggregated_results(
    dataset='FordA', 
    model_name='ResidualCNN', 
    exp_name='',
    reg=True,  
    metr='F_EFF_CONC',
    metr_max=True,
    k=1,
    vis = False,
    params=['ACC', 'PROB_HID'],
    figsize=(20, 30),
    save_pic=False):

    all_df = combined_df(dataset=dataset, model_name=model_name, reg=reg, exp_name=exp_name)

    grouped = all_df.groupby(['eps', 'alpha'])
    result = grouped.apply(lambda x: x.nlargest(k, metr, keep='last') if metr_max else x.nsmallest(k, metr, keep='last'))


    if vis:
        
        fig, ax = plt.subplots(result['alpha'].nunique(), result['eps'].nunique(), figsize=figsize, sharey=True)
        fig.suptitle(f'{dataset}_{model_name}')

        all_alps = result['alpha'].unique()
        all_eps = result['eps'].unique()

        path = get_path(dataset=dataset, model_name=model_name, reg=reg, exp_name=exp_name)

        for i, row in result.iterrows():

            alp = int(row['alpha']) if row['alpha'].is_integer() else row['alpha']
            eps = row['eps']

            file_path = path + '/' + f'aa_res_{dataset}_{int(row["model_id"])}_alpha={alp}.csv'

            curr_df = pd.read_csv(file_path, index_col=0)
            curr_df.columns = [column.strip(' ') for column in curr_df.columns]

            curr_df = curr_df[curr_df['eps']==eps].set_index('step_id')

            eps_id = np.where(all_eps==eps)[0][0]
            alp_id = np.where(all_alps==alp)[0][0]

            ax[alp_id, eps_id].plot(curr_df[params])
            ax[alp_id, eps_id].axvline(int(row["step_id"]), ls='--', alpha=0.2, color='black')

            ax[alp_id, eps_id].set_title(f'eps={eps}, alpha={alp}, {metr}={row[metr]}, step={int(row["step_id"])}')
            ax[alp_id, eps_id].legend(params)
        
        fig.tight_layout()
        fig.subplots_adjust(top=0.95)

        if save_pic:
            plt.savefig(f'../{dataset}_{model_name}.jpeg', dpi=120)

    return result

In [13]:
aggregated_results(dataset='Coffee', model_name='PatchTST', exp_name="MODE", vis=False, params=['ACC', 'PROB_HID', 'ACC_DISC'])

ACC     ROC      PR      F1     EFF        L1  \
eps   alpha                                                           
0.005 0.001   16   0.4286  0.4359  0.4360  0.4667  0.5333   17.6482   
      0.010   641  0.3214  0.3000  0.4643  0.0000  1.0000   29.4750   
      0.100   356  0.5000  0.4667  0.4643  0.0000  1.0000   50.2082   
      1.000   203  0.6071  0.5769  0.5467  0.2667  0.7333   53.5750   
      10.000  815  0.6786  0.6538  0.6291  0.4706  0.5294   53.8953   
      100.000 509  0.7143  0.6923  0.6703  0.5556  0.4444   53.9639   
0.030 0.001   54   0.4286  0.4308  0.4346  0.4286  0.5714   19.4871   
      0.010   668  0.3571  0.3385  0.4396  0.1000  0.9000   29.5757   
      0.100   407  0.4286  0.4000  0.4643  0.0000  1.0000  250.5277   
      1.000   254  0.5000  0.4667  0.4643  0.0000  1.0000  273.2527   
      10.000  866  0.5714  0.5385  0.5055  0.1429  0.8571  287.7084   
      100.000 560  0.5714  0.5385  0.5055  0.1429  0.8571  289.5941   
0.100 0.001   104  0.1429  0.1333  0.4643  0.0000  1.0000   35.0357   
      0.010   716  0.5000  0.4718  0.4542  0.1250  0.8750   31.7786   
      0.100   458  0.3571  0.3333  0.4643  0.0000  1.0000  368.6785   
      1.000   305  0.5000  0.4667  0.4643  0.0000  1.0000  367.6142   
      10.000  917  0.5714  0.5385  0.5055  0.1429  0.8571  364.4714   
      100.000 611  0.5714  0.5385  0.5055  0.1429  0.8571  358.3928   

                   ACC_CORRECT  ACC_ORIG_ADV  ROUGHNESS  ROUGHNESS_NORM  \
eps   alpha                                                               
0.005 0.001   16        0.4286        0.4286     0.0934          2.4532   
      0.010   641       0.3214        0.3214     0.1656          4.3504   
      0.100   356       0.5000        0.5000     0.3086          8.1058   
      1.000   203       0.6071        0.6071     0.3527          9.2648   
      10.000  815       0.6786        0.6786     0.3690          9.6917   
      100.000 509       0.7143        0.7143     0.3716          9.7624   
0.030 0.001   54        0.4286        0.4286     0.1055          2.7725   
      0.010   668       0.3571        0.3571     0.1724          4.5276   
      0.100   407       0.4286        0.4286     1.7302         45.4484   
      1.000   254       0.5000        0.5000     1.9112         50.2044   
      10.000  866       0.5714        0.5714     2.0888         54.8699   
      100.000 560       0.5714        0.5714     2.0913         54.9344   
0.100 0.001   104       0.1429        0.1429     0.1982          5.2073   
      0.010   716       0.5000        0.5000     0.2171          5.7029   
      0.100   458       0.3571        0.3571     2.6122         68.6186   
      1.000   305       0.5000        0.5000     2.6298         69.0791   
      10.000  917       0.5714        0.5714     2.6525         69.6751   
      100.000 611       0.5714        0.5714     2.6037         68.3945   

                   PROB_HID  ACC_DISC  F1_DISC  ROC_AUC_DISC    CONC  \
eps   alpha                                                            
0.005 0.001   16     0.0123    0.5000   0.0000        0.3099  0.6901   
      0.010   641    0.0794    0.5357   0.1333        0.2181  0.7819   
      0.100   356    0.0000    0.5000   0.0000        0.0000  1.0000   
      1.000   203    0.0000    0.5000   0.0000        0.0000  1.0000   
      10.000  815    0.0000    0.5000   0.0000        0.0000  1.0000   
      100.000 509    0.0000    0.5000   0.0000        0.0000  1.0000   
0.030 0.001   54     0.0096    0.5000   0.0000        0.2258  0.7742   
      0.010   668    0.0188    0.5000   0.0000        0.1416  0.8584   
      0.100   407    0.0000    0.5000   0.0000        0.0000  1.0000   
      1.000   254    0.0000    0.5000   0.0000        0.0000  1.0000   
      10.000  866    0.0000    0.5000   0.0000        0.0000  1.0000   
      100.000 560    0.0000    0.5000   0.0000        0.0000  1.0000   
0.100 0.001   104    0.2615    0.6250   0.4000        0.5497  0.4503   
      0.010   716   

In [14]:
res = pd.DataFrame()

for model in [ 'ResidualCNN', 'RNNA', 'TST', 'S4', 'PatchTST']:
    for ds in ['Coffee', 'GunPoint', 'PowerCons', 'Wafer']:
        try:
            aa = aggregated_results(dataset=ds, model_name=model, exp_name="MODE")
            aa['model'] = model
            aa['dataset'] = ds

            res = pd.concat([res, aa], axis=0)
        except:
            print(ds, model)

Wafer S4
Wafer PatchTST


In [15]:
res.groupby(['dataset', 'model']).max().reset_index().pivot(index='model', columns='dataset', values='F_EFF_CONC')

dataset,Coffee,GunPoint,PowerCons,Wafer
model,,,,
PatchTST,1.0,0.8248,0.711400,NaN
RNNA,1.0,0.4347,0.252500,0.1039
ResidualCNN,1.0,0.4716,0.663100,0.7080
S4,1.0,1.0000,0.397600,NaN
TST,1.0,0.3554,0.926621,0.5989


In [10]:
tmp = res.groupby(['dataset', 'model'])['F_EFF_CONC'].idxmax().to_frame()

tmp['eps'] = tmp['F_EFF_CONC'].apply(lambda x: x[0])
tmp['alpha'] = tmp['F_EFF_CONC'].apply(lambda x: x[1])
tmp['F_EFF_CONC'] = res.groupby(['dataset', 'model'])['F_EFF_CONC'].max()

In [11]:
tmp.reset_index().pivot(index='model', columns='dataset')

F_EFF_CONC                               eps                     \
dataset         Coffee GunPoint PowerCons   Wafer Coffee GunPoint PowerCons   
model                                                                         
PatchTST           1.0   0.8248  0.779100     NaN  0.005    0.100      0.10   
RNNA               1.0   0.4347  0.252500  0.1039  0.030    0.100      0.03   
ResidualCNN        1.0   0.4716  0.948376  0.7080  0.005    0.005      0.03   
S4                 1.0   1.0000  0.397600     NaN  0.030    0.030      0.10   
TST                1.0   0.3554  0.926621  0.5989  0.005    0.005      0.03   

                    alpha                            
dataset      Wafer Coffee GunPoint PowerCons  Wafer  
model                                                
PatchTST       NaN    0.1    1.000     0.100    NaN  
RNNA         0.030    1.0   10.000     0.100   0.10  
ResidualCNN  0.005   10.0    1.000     0.010  10.00  
S4             NaN    1.0    1.000    10.000    NaN  
TST          0.030    0.1    0.001     0.001   0.01

In [12]:
best_metr = tmp.reset_index().pivot(index='model', columns='dataset')

datasets = best_metr['F_EFF_CONC'].columns
models = best_metr.index

metr = ['ACC', 'PROB_HID', 'ACC_DISC']

save = True

In [10]:
fig, ax = plt.subplots(len(datasets), len(models), figsize=(35, 20))

for i, dataset in enumerate(datasets):
    for j, model in enumerate(models):

        eps = best_metr['eps'].loc[model, dataset]

        alpha = best_metr['alpha'].loc[model, dataset]
        alpha = int(alpha) if alpha.is_integer() else alpha

        path = get_path(dataset, model, exp_name='MODE')

        try:
            curr_res = pd.read_csv(path + '/' + f'aa_res_{dataset}_0_alpha={alpha}.csv', index_col=0)
            curr_res.columns = [col.strip(' ') for col in curr_res.columns]
            
            curr_res = curr_res[curr_res['eps'] == eps].set_index('step_id')
            ax[i, j].plot(curr_res[metr])
            ax[i, j].set_title(f'{dataset}  {model}  F_EFF_CONC={curr_res["F_EFF_CONC"].max()}')
            ax[i, j].legend(metr)

        except:
            print(model, dataset, alpha)
        
        fig.tight_layout()
        fig.subplots_adjust(top=0.95)

if save:
    fig.savefig('../img.jpeg', dpi=200)

NameError: name 'datasets' is not defined